## A simple Flask Web-app to test LLM model

In [ ]:
# install tools
%pip install accelerate

In [ ]:
# initialize LLM model
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "/content/drive/MyDrive/code/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2"
#model_path = "google/flan-t5-base"
#tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path, device_map="auto")


In [25]:
# Get AI response from user input
#  
def get_flan_t5_text( str_promp ):
  #input_text = "translate English to German: How old are you?"
  print(str_promp)
  input_text = str_promp
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
  outputs = model.generate(input_ids)
  return tokenizer.decode(outputs[0])

In [26]:
# code section for web-app (Flask)
#   part 1: reveal public url

from google.colab import output

output.serve_kernel_port_as_window(8084)

<IPython.core.display.Javascript object>

In [ ]:
#   part 2: web-app page

from flask import Flask, request

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
  str_input = str_t5 = ''
  if (request.method == "POST"):
    str_input = request.form.get("get_prompt")
    str_t5 = get_flan_t5_text( str_input )

  str_page = """
    <html>
      <body>
        <form action='/' method='post'>
          Put Prompt: <input name='get_prompt' value='{}'> <button type='submit'>Submit</button><br>
          <hr>
          Response: <br>
          {}
        </form>
      </body>
    </html>
  """
  str_page = str_page.format(str_input, str_t5)
  return str_page

app.run(port=8084)